In [1]:
import requests
from bs4 import BeautifulSoup
import pandas  as  pd

In [2]:
url = "https://repositorio.lamolina.edu.pe/recent-submissions?offset="

# Realizar una petición HTTP a la URL
response = requests.get(url)

# Analizar el contenido HTML de la página usando Beautiful Soup
soup = BeautifulSoup(response.content, "html.parser")

# Crea el limite superior del for tomando en cuenta la cantidad de tesis expuestas en  la pagina
limsup=int((soup.find("p",{"class":"pagination-info"})).text[-4:])/20+1

In [3]:
hrefs=[]
# Iterar sobre todos los valores de i desde 0 hasta 100
for i in range(0, int(limsup)):
    # Crear la URL de la página a scrappear
    url = "https://repositorio.lamolina.edu.pe/recent-submissions?offset=" + str(i*20)

    # Realizar una petición HTTP a la URL
    response = requests.get(url)

    # Analizar el contenido HTML de la página usando Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extraer todos los enlaces de la página
    links = soup.find_all("h4",{"class":"artifact-title"})


    # Iterar sobre todos los enlaces extraídos y mostrarlos por pantalla
    for link in links:
        hrefs.append("https://repositorio.lamolina.edu.pe"+link.find("a").get("href")+"?show=full")

In [4]:
institucion=[]
titulo=[]
autor=[]
grado=[]
asesor=[]
resumen=[]
anio=[]
vacio=""

In [5]:
for i in range(len(hrefs)):
    response=requests.get(hrefs[i])
    
    # Analizar el contenido HTML de la página usando Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extraer la tabla que deseas
    table = soup.find("table", {"class": "ds-includeSet-table detailtable table table-striped table-hover"})

    # Convertir la tabla en un DataFrame de Pandas
    df = pd.read_html(str(table),index_col=0)[0]
    
    # Nos quedamos con la parte de la tabla que nos importa
    df=df[:][1]
    
    # Scrappeo de datos
    if "dc.publisher" in df:
        institucion.append(df["dc.publisher"])
    else:
        institucion.append(vacio)
    if "dc.title" in  df:
        titulo.append(df["dc.title"])
    else:
        titulo.append(vacio)
    
    if "dc.contributor.author" in df:
        if type(df["dc.contributor.author"])==object :
            autor.append(df["dc.contributor.author"][:].apply(lambda x: x).tolist())
        else :
            autor.append(df["dc.contributor.author"])
    else:
        autor.append(vacio)
    
    if "dc.type" in df:
        grado.append(df["dc.type"])
    else:
        grado.append(vacio)
        
    if "dc.contributor.advisor" in df:
        if type(df["dc.contributor.advisor"])==object :
            asesor.append(df["dc.contributor.advisor"].apply(lambda x: x).tolist())
        else :
            asesor.append(df["dc.contributor.advisor"])
    else:
        asesor.append(vacio)
    
    if "dc.description.abstract" in df:
        resumen.append(df["dc.description.abstract"][0])
    else:
        resumen.append(vacio)
    
    if "dc.date.issued" in df:
        anio.append(df["dc.date.issued"])
    else:
        anio.append(vacio)

In [6]:
datosexportar = pd.DataFrame(list(zip(anio,institucion,titulo,autor,grado,asesor,resumen)),columns =["Año","Institución","Titulo","Autor","Grado","Asesor","Resumen"])

In [8]:
datosexportar.to_csv("repositorio_agraria_tesis.csv", sep=',')